In [1]:
import os
os.chdir('ML_training_01/')

FileNotFoundError: [Errno 2] No such file or directory: 'ML_training_01/'

In [2]:
%%bash 
ls

Course_1.ipynb
clase1.py


In [ ]:
%%bash
mkdir ML_training_01
chown -R jupyter:jupyter /home/jupyter/ML_training_01

FileNotFoundError: [Errno 2] No such file or directory: 'ML_training_01/'

In [8]:
%%bash
# Iniciar un repositorio
git init
# revisar que commits tenemos para cargar
git status
# Seleccionamos que cosas queremos cargar al repositorio
git add Course_1.ipynb
#Se crea el comit dentro de nuestro repositorio local
git commit -m "Se argega el proyecto"

Reinitialized existing Git repository in /home/jupyter/ML_training_01/.git/
On branch main
Your branch is up to date with 'origin/main'.

Changes not staged for commit:
  (use "git add <file>..." to update what will be committed)
  (use "git checkout -- <file>..." to discard changes in working directory)

	modified:   Course_1.ipynb

Untracked files:
  (use "git add <file>..." to include in what will be committed)

	.gitignore
	.ipynb_checkpoints/.gitignore-checkpoint
	".ipynb_checkpoints/Capacitaci\303\263n  Introducci\303\263n ML-checkpoint.ipynb"
	.ipynb_checkpoints/clase1-checkpoint.py
	"Capacitaci\303\263n  Introducci\303\263n ML.ipynb"
	FONDO_EPM_GCP.xlsx
	Fondo_PP_GCP.xlsx
	ML_train_functions_gcp.py
	clase1.py
	data/

no changes added to commit (use "git add" and/or "git commit -a")
[main 91d4703] Se argega el proyecto
 1 file changed, 276 insertions(+), 42 deletions(-)


In [9]:
%%bash
#Subiremos esto a nuestra rama principal
git branch -M main
#Cargaremos al proeycto ML_training_01.git
git remote rm origin
git remote add origin https://github.com/JAPJ182/ML_training_01.git

In [3]:
%%bash
# Iniciar un repositorio
git init
# revisar que commits tenemos para cargar
git status
# Seleccionamos que cosas queremos cargar al repositorio
git add Course_1.ipynb
#Se crea el comit dentro de nuestro repositorio local
git commit -m "Se argega el proyecto"
#Subiremos esto a nuestra rama principal
git branch -M main
#Cargaremos al proeycto ML_training_01.git
git remote rm origin
git remote add origin https://github.com/JAPJ182/ML_training_01.git
#git remote add origin https://github.com/<JAPJ182>:<*Neftigaza2021*>/ML_training_01.git
git config --global user.email japj182@hotmail.com
git config --global user.name  "JAPJ182"
git push -u origin main 

Reinitialized existing Git repository in /home/jupyter/ML_training_01/.git/
On branch main
Your branch is ahead of 'origin/main' by 1 commit.
  (use "git push" to publish your local commits)

Changes not staged for commit:
  (use "git add <file>..." to update what will be committed)
  (use "git checkout -- <file>..." to discard changes in working directory)

	modified:   Course_1.ipynb

Untracked files:
  (use "git add <file>..." to include in what will be committed)

	.gitignore
	.ipynb_checkpoints/.gitignore-checkpoint
	.ipynb_checkpoints/clase1-checkpoint.py
	clase1.py

no changes added to commit (use "git add" and/or "git commit -a")
[main df2e4eb] Se argega el proyecto
 1 file changed, 144 insertions(+), 1 deletion(-)


fatal: could not read Username for 'https://github.com': No such device or address


CalledProcessError: Command 'b'# Iniciar un repositorio\ngit init\n# revisar que commits tenemos para cargar\ngit status\n# Seleccionamos que cosas queremos cargar al repositorio\ngit add Course_1.ipynb\n#Se crea el comit dentro de nuestro repositorio local\ngit commit -m "Se argega el proyecto"\n#Subiremos esto a nuestra rama principal\ngit branch -M main\n#Cargaremos al proeycto ML_training_01.git\ngit remote rm origin\ngit remote add origin https://github.com/JAPJ182/ML_training_01.git\n#git remote add origin https://github.com/<JAPJ182>:<*Neftigaza2021*>/ML_training_01.git\ngit config --global user.email japj182@hotmail.com\ngit config --global user.name  "JAPJ182"\ngit push -u origin main \n'' returned non-zero exit status 128.

In [7]:
%%writefile ML_train_functions_gcp.py
import chardet
import pandas as pd
import time
import re
from google.cloud import bigquery
# Método del codo para averiguar el número óptimo de clusters

def optimal_cluster (x):
    from sklearn.cluster import KMeans
    wcss = []
    for i in range(1, 11):
        kmeans = KMeans(n_clusters = i, init = "k-means++", max_iter = 300, n_init = 10, random_state = 0)
        kmeans.fit(X)
        wcss.append(kmeans.inertia_)

    plt.plot(range(1,11), wcss)
    plt.title("Método del codo")
    plt.xlabel("Número de Clusters")
    plt.ylabel("WCSS(k)")
    plt.show()

def feature_eng(tabla, X, Y, X_num , X_Cat ):
    label =  tabla[Y]
    features = tabla[ X ]
    #####
    numeric_feature = X_num
    lista = X_Cat
    # Comentarios
    features = pd.get_dummies(features, columns=lista, drop_first=False)       
    ##
    from sklearn.model_selection import train_test_split
    train_features, test_features, train_label, test_label = train_test_split(features, label, test_size = 0.25, random_state = 0)
    return  train_features, test_features, train_label, test_label


class load_simple_file_class:
    def __init__(self,
                 path = "" ,
                 delimit = "",
                 
                 ):  
        
        # Instance Variable 
        self.tabla =path
        self.formato_ = path.split('.')[-1].upper()
        self.table_name = path.split('/')[-1].upper().split('.')[0]  
        self.delimit = delimit
        
    def econde_tabla(self): 
        if (self.formato_.upper() == 'CSV') or (self.formato_.upper() == 'TXT'):        
            import chardet

            with open(self.tabla, 'rb') as rawdata:
                result = chardet.detect(rawdata.read(10000))

            return result['encoding']
        else:
            pass

    def delimitador(self):
        if self.delimit == "":
            x = 15
            if (self.formato_.upper() == 'CSV') or (self.formato_.upper() == 'TXT'):
                self.encode = self.econde_tabla( )
                if pd.read_csv(self.tabla, sep=',', encoding= self.encode , nrows = 1).shape[1]>x:
                    sep_ = ','
                elif pd.read_csv(self.tabla, sep=';', encoding= self.encode, nrows = 1).shape[1]>x:
                    sep_ = ';'
                elif pd.read_csv(self.tabla, sep='|', encoding= self.encode, nrows = 1).shape[1]>x:
                    sep_ = '|'
                elif pd.read_csv(self.tabla, sep=':', encoding= self.encode, nrows = 1).shape[1]>x:
                    sep_ = ':'  
                else:
                    pass  
                return sep_          
            else:
                pass
        else:
            sep_ = self.delimit
            return sep_  
         

    def types_dict(self):
        if self.formato_.upper() == 'CSV'  or self.formato_.upper() == 'TXT' :
            col_names = pd.read_csv(self.tabla,sep= self.delimitador(), nrows=0, encoding =  self.econde_tabla() ).columns
            types_dict_ = { }
            types_dict_.update({col: str for col in col_names  })
            return types_dict_
        elif self.formato_.upper() == 'XLS' or self.formato_.upper() == 'XLSX':
            col_names = pd.read_excel(self.tabla, nrows = 1 )
            types_dict_ = { }
            types_dict_.update({col: str for col in col_names  })
            return types_dict_
        elif self.formato_.upper() == 'XLSB' :
            col_names = pd.read_excel(self.tabla, nrows = 1 )
            types_dict_ = { }
            types_dict_.update({col: str for col in col_names  })
            return types_dict_
        else:
            pass
        
        print('reading parameters have been set');
 
    def read_tables(self):
        self.start = time.time()
        ### 
        if (self.formato_.upper() == 'ZIP'  or self.formato_.upper() == 'RAR'):
            pass
            "i need to get the table and then read, normalize colnames and then sent to bq"
        
        elif (self.formato_.upper() == 'CSV'  or self.formato_.upper() == 'TXT' ) :
            try:
                Data  =  pd.read_csv(self.tabla, sep=self.delimitador(), encoding= self.econde_tabla(),   warn_bad_lines=False, error_bad_lines=False )
            except:
                Data = pd.read_csv(self.tabla, sep=self.delimitador(), encoding= self.econde_tabla(),engine = 'python',    warn_bad_lines=False, error_bad_lines=False )
            
            return Data
        
        elif (self.formato_.upper() == 'XLS' or self.formato_.upper() == 'XLSX' ):    
            
            Data  = pd.read_excel(self.tabla, converters = self.types_dict())
            
            return Data
        else:
            pass
        
    def read_table(self):
        start = time.time()
        def normalize_(s):
            replacements = (
                (" ","_")    ,("á", "a"),("à", "a"),
                ("é", "e"),("è", "e"),
                ("í", "i"),('ï»¿',''), ("ì","i"), ("ó","o"),  ("ò","o"),  ("ö","o"),
                ("ú", "u"),("ù", "u"), ("ü", "u"), ("û", "u"),  
                ("ñ", "n"), ("*",""),
                ######################################################                
                ('├í','a'),  ("├®","e"), ('├¡','i'), ('├│','o'),
                ('├║','u'), ('├▒','ni'),  
                ('├ü','A'),('├Ç','A'),('├ç','A'),(":","_"),
                ('├ë','E'),("├ë","E"),("├ê","E"),("╔","E"),
                ('├ì','I'),("├î","I"),('├û','I'),('├Å','I'),
                ('├ô','O'),('├ï','O'),('├è','O'),('@', 'O'),
                ('├Ö','U'),
                ('├æ','Ñ'),('├É','ni'),('├æ','Ñ'),('Ð','ni'),
                ('├Ü','U'),
                ('├û','Í'  )  , ("\n","_"),
                ###############################################################
                (".", "_"), ("/","_"),("\\", "_"), (")", "_"),("(", "_"),('>',''), ('<',''),
                ("\¿", ""),("\?", ""),(" ", "_"),(",", "_"),("__", "_"),("-", ""),
                ("/¿", ""),("/?", ""),
                ("0_", "X0_"), ("1_", "X1_"),("2_", "X2_"),("3_", "X3_"), ("4_", "X4_"),
                ("5_", "X5_"),("6_", "X6_"),("7_", "X7_"),("8_", "X8_"),("9_", "X9_"),
                ("__", "_"),  (" ", "_"),  (" ", "_")
             )
            for a, b in replacements:
                
                s = s.replace(a, b).replace(a.upper(), b.upper()).strip().lstrip('1').lstrip('2').lstrip('3').lstrip(
                        '4').lstrip('5').lstrip('6').lstrip('7').lstrip('8').lstrip('9').lstrip('0').lstrip(
                        '_').rstrip('_').strip(' ').rstrip().replace("á", "a").replace("é", "e").replace("í", "i").replace("ó", "o").replace("ú", "u") 
                R = re.sub(r"[^a-zA-Z0-9]","",s.upper()) 
            return R
        
        nombre_columnas = []
        Data = self.read_tables()
         
        for i in range(len(Data.columns)):
            lista  = normalize_(pd.DataFrame(list(Data.columns), columns =['columnas']).columnas[i].upper() )[0:20] 
            nombre_columnas.append(lista)
 
 
        
        Data.columns = nombre_columnas
        schema_ = []
        for i in Data.columns:
            schema_line   =   bigquery.SchemaField("""{}""".format(i), 'STRING' , mode = 'NULLABLE')
            schema_.append(schema_line)
        print('esquema creado correctamente')
        
        return [Data, schema_, nombre_columnas]
        


Overwriting ML_train_functions_gcp.py
